# Sample Hangul RNN

In [1]:
# -*- coding: utf-8 -*-
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import string
import argparse
import time
import os
import hgtk

from six.moves import cPickle
from TextLoader import *

print ("Packages Imported")

Packages Imported


# Load dataset using TextLoader

In [2]:
data_dir    = "data/nine_dreams"
batch_size  = 50
seq_length  = 50
data_loader = TextLoader(data_dir, batch_size, seq_length)
# This makes "vocab.pkl" and "data.npy" in "data/nine_dreams"   
#  from "data/nine_dreams/input.txt" 
vocab_size = data_loader.vocab_size
vocab = data_loader.vocab
chars = data_loader.chars
print ( "type of 'data_loader' is %s, length is %d" 
       % (type(data_loader.vocab), len(data_loader.vocab)) )
print ( "\n" )
print ("data_loader.vocab looks like \n%s " %
       (data_loader.vocab))
print ( "\n" )
print ( "type of 'data_loader.chars' is %s, length is %d" 
       % (type(data_loader.chars), len(data_loader.chars)) )
print ( "\n" )
print ("data_loader.chars looks like \n%s " % (data_loader.chars,))

loading preprocessed files
type of 'data_loader' is <class 'dict'>, length is 76


data_loader.vocab looks like 
{'"': 28, 'ㅢ': 25, 'ㅛ': 38, '1': 58, '9': 64, '\n': 19, 'ㄷ': 13, 'ㄾ': 70, 'ㅈ': 15, '(': 45, 'ㅄ': 43, '8': 68, 'ㅏ': 3, '>': 75, '5': 63, 'ㅠ': 37, ')': 46, 'ㅜ': 16, '6': 59, '\x1a': 72, 'ㅀ': 51, 'ㅗ': 9, 'ㅌ': 30, '3': 66, '0': 73, 'ㅃ': 47, 'ㅎ': 14, 'ㅁ': 12, 'ㅣ': 6, 'ㅝ': 36, 'ㄴ': 4, ' ': 1, 'ㄻ': 55, 'ㅖ': 39, 'ㅓ': 11, '.': 24, 'ㅚ': 33, 'ㅉ': 40, 'ㅘ': 26, 'ㄱ': 7, 'ㅂ': 17, 'ㅆ': 22, '!': 52, ':': 57, 'ㄵ': 50, 'ㄺ': 48, 'ㄳ': 60, 'ㅔ': 20, 'ㅊ': 23, '4': 67, 'ㄿ': 56, ',': 27, 'ㅑ': 34, 'ㄶ': 44, '7': 71, 'ㅋ': 42, 'ㄸ': 32, '-': 65, 'ㅍ': 31, 'ㄲ': 29, 'ㅙ': 53, 'ㅞ': 61, '2': 62, 'ㄼ': 54, 'ㅕ': 18, 'ㅒ': 74, 'ㅡ': 8, '_': 69, '?': 41, 'ㅇ': 2, 'ᴥ': 0, 'ㅐ': 21, "'": 49, 'ㅟ': 35, 'ㅅ': 10, 'ㄹ': 5} 


type of 'data_loader.chars' is <class 'tuple'>, length is 76


data_loader.chars looks like 
('ᴥ', ' ', 'ㅇ', 'ㅏ', 'ㄴ', 'ㄹ', 'ㅣ', 'ㄱ', 'ㅡ', 'ㅗ', 'ㅅ', 'ㅓ', 'ㅁ', 'ㄷ', 'ㅎ', 'ㅈ', 'ㅜ', 'ㅂ', 'ㅕ', '\n', 'ㅔ', 'ㅐ', 

# Define Network

In [3]:
rnn_size   = 512
num_layers = 3
grad_clip  = 5.

_batch_size = 1
_seq_length = 1

vocab_size = data_loader.vocab_size

with tf.device("/cpu:0"):
    # Select RNN Cell
    unitcell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([unitcell] * num_layers)
    # Set paths to the graph 
    input_data = tf.placeholder(tf.int32, [_batch_size, _seq_length])
    targets    = tf.placeholder(tf.int32, [_batch_size, _seq_length])
    initial_state = cell.zero_state(_batch_size, tf.float32)

    # Set Network
    with tf.variable_scope('rnnlm'):
        softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
        softmax_b = tf.get_variable("softmax_b", [vocab_size])
        with tf.device("/cpu:0"):
            embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
            inputs = tf.split(tf.nn.embedding_lookup(embedding, input_data), _seq_length, 1)
            inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
            
    # Loop function for seq2seq
    def loop(prev, _):
        prev = tf.nn.xw_plus_b(prev, softmax_w, softmax_b)
        prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
        return tf.nn.embedding_lookup(embedding, prev_symbol)
    # Output of RNN 
    outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state
                                , cell, loop_function=None, scope='rnnlm')
    output = tf.reshape(tf.concat(outputs, 1), [-1, rnn_size])
    logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
    # Next word probability 
    probs = tf.nn.softmax(logits)
    # Define LOSS
    loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], # Input
        [tf.reshape(targets, [-1])], # Target
        [tf.ones([_batch_size * _seq_length])], # Weight 
        vocab_size)
    # Define Optimizer
    cost = tf.reduce_sum(loss) / _batch_size / _seq_length
    final_state = last_state
    lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    _optm = tf.train.AdamOptimizer(lr)
    optm = _optm.apply_gradients(zip(grads, tvars))

print ("Network Ready")

Network Ready


In [4]:
# Sample ! 
def sample( sess, chars, vocab, __probs, num=200, prime=u'ㅇㅗᴥㄴㅡㄹᴥ '):
    state = sess.run(cell.zero_state(1, tf.float32))
    _probs = __probs
    prime = list(prime)
    for char in prime[:-1]:
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]
        feed = {input_data: x, initial_state:state}
        [state] = sess.run([final_state], feed)

    def weighted_pick(weights):
        weights = weights / np.sum(weights) 
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime
    char = prime[-1]
    for n in range(num):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]
        feed = {input_data: x, initial_state:state}
        [_probsval, state] = sess.run([_probs, final_state], feed)
        p = _probsval[0]
        sample = int(np.random.choice(len(p), p=p))
        # sample = weighted_pick(p)
        # sample = np.argmax(p)
        pred = chars[sample]
        ret += pred
        char = pred
    return ret
print ("sampling function done.")

sampling function done.


# Sample

In [5]:
save_dir = 'data/nine_dreams'
prime = hgtk.text.decompose(u"누구 ")

print ("Prime Text : %s => %s" % (hgtk.text.compose(prime), "".join(prime)))
n = 2000

sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state(save_dir)

# load_name = u'data/nine_dreams/model.ckpt-0'
# load_name = u'data/nine_dreams/model.ckpt-99000'
load_name = tf.train.latest_checkpoint(save_dir)

print (load_name)

if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, load_name)
    sampled_text = sample(sess, chars, vocab, probs, n, prime)
    #print ("")
    print (u"SAMPLED TEXT = %s" % "".join(sampled_text))
    print ("")
    print ("-- RESULT --")
    print (hgtk.text.compose("".join(sampled_text)))

누
구
 
Prime Text : 누구  => ㄴㅜᴥㄱㅜᴥ 
data/nine_dreams/model.ckpt-0
SAMPLED TEXT = ㄴㅜᴥㄱㅜᴥ ?ㅆ>ㅀ79ㄼㅖㅝ5:6ㅜ-8>ㅂ1ㄳㄲㅃ0ㄻㄾ1ㅠㅙㅊ5ㅀ)ㄵ2ㅕ(ㄷ4(ㅊㅎ_ᴥᴥㅌ>ㅢㅃㅢㅈㅢㅞ 44ㅜㅟ:ㅜㅑㅒㄺㅈㅡㄵ72ㅍㅡㅙㅀ0ㅊㅅㅜㅆㅍㅛㄹㄱㅃ>ㄳ_. ㅡㅁㅅㅌ?ㄿㄻㅑ
ㅣㄸㅋㅏㅅㅊ2ㅄㅠㄵㅐㅞㄾㅏㄱㄵㄴ?_ㄿㅎㅋ2ㅢ8ㅚㅘㅎㄶㅀㄲㅅㅍ6ㅔㄶㅋㅍㅈㅕ'7ㅣ72ㅜㅠㅀㅣ>ㄳㅉㅉ,ㅍㅅᴥㄸㅡㅟ5ㅎ5 3ㄳ(4ㅓ4ㅡㄳㅀㅙㄻㅘ24ㅈㅊ!ㅍㄷㅕㄶㅖ>ㅆㄲ6ㅑ50!.ㅠㅀㄿㅊㅍㅇㅜㅈ(ㅜㅠ!ㄻ"4ㅕ_')ㅖㅒㅂㄲ(ㅐ.ㅖㅍㄳㅢㄷ!"ㅓ?ㅈㅎㄺㅁ,-ㅗㅠㅅㅏㅡ.ㅠㄲㅁ.ㅏㄸㅙㅊ 3> ㅠㄸ896ㅡ
ㅣㅏ!25ㅀ5ㅏㅋㅞㅃㅚㅂㅊㅌ5ㅠㄹㅉ.0ㄼㅔㅟㅕ9ㄳ"ㅄㄶㅒㄻㅀㅞㅗ3ㅇㅙㅑㅍㅄㄷㅑ.ㄿㄵ6ㅈ,ㅠㅑ?ㅑㄾㅅㅄᴥㅞㅉㅌㅌ'ㅀㅂㅙㄸㄿㅉ0ㅜㄻᴥ>ㅂᴥ!
)0ㅅㄱㅛᴥㅋ>ㅉ4ㄾㅃ_ㅁㅈㄶ8ㅄㄵ!ㅓㅎㄱ_ㅔ.ㅆㅇㅍㅘㅅ2ㅏㅌㅁㅡㅁ!ㄹ92ㅣㄻ7)4ㅃ_ㅊ1ㅁㅣㅎㅒㄼㅀㅔㅕㅅㅔ- 'ㅜㄷㅙㄶㄺ_ㄵㅀ?ㅣ-ㄵ(.ㅝ""ㅒㄸㅋ"ㅝㄲㅡ0ㅁㅜㅙ7ᴥ0!ㅢ 1ㅢ !ㅖㄻㅕㅌ ㄾㄾㄸㅆㅃ>ㄳ7ㅠᴥㅚㅠㄹㅞㅉㅆㄺㄿㅈㅒㅉㄶ?ㅙㄻㅞㅉ
ㅃㅉㅒ(>ㅖㅆㅕㄷ)ㄺㅋㄾ6ㅑ'ㄺ.16.ㄿㅟㅋㅃㅞㅕㅗㅉㅀ
?ㅣㅃㅌㅎㅍㄱㅑ ㅑㅙㅎㄿ8ㄶㄿㄸㅜㅛᴥㄿㅂㅏㅡㅄㅞㄾㄸ8ㅕ5ㅙㅊ
7ㄻ4ㄱㄳㄴ,ㅅㅔㅢ42ㄸㅖㄵㅌ'ㅢ9ㅈㅞ!ㄻ?4( ㄼㄾ1!ㅊ4ㅜㅔㅔㅑㅚㄷ:1ㅚㅈㅣㅀㅛ ㅓㅉ>8ㄷㅅㅓㄳ2ㄾ
ㅏᴥㅊㅊㅇㅍㅕ6ㅂㅑ5ㅐ8ㅕ-(ㅑ8ㄴㄺ>ㅙㄷㅞㅌㅀㄼㅔᴥㅈ,ㄺ3ㅚㄶㅡㅞㅀㅛㅞㅚㅠㅡㄳㅇ4ㅘㅔㅍㄱㅍㅜㅉ'ㄳㅣ6ㅆㄻ
ㄳㅟㄲ6ㄾ)ㄸ)ㄺㅋ"3ㅎㅃㅘ1ㅗㅜ-ㅅㅅㅇㅙㄸㅠㅟ!ㄼ6ᴥㅚㅟㅐㄹ8ㅎㅆ0ㅌㅚㄹ3ㅐㄿㄾㅔ18ㄿㅛ28.1'ㄲ6ㅍㅒㅐㄻㅏㅑㅈ6?66ㅟ>ㅟㄴㅈ?ㅀㅉㅏ7ㅚ3ㅓㅔㅍ5ㅗㅏㅋ:ㅌㄳㅛㅌ6ㅅㅡㅉㅅㅣㄵㄿ"ㅇ:ㅊㄲㅂ8"('ㅠㅁㅊㄱㅙㅃㅔㅑㄼㅚㅂㄸㄺㅛㅎㅚㄲㄲㄸ3ᴥㅚㅅ7ㅞㅅㅖㅓㅠㅑㄻㅉㅁㅝㅑㅕㄹㅆᴥㄴㅉ9"ㄺㅋ25",ㄺㅚㅂᴥㅟ5'8:ㅣㅓㅟ.ㅓ0ㅂㅛ-9(ㅋㅒㄴㄼㄵ-ㅙ_,ㅆㄾㅠ,ㅈᴥㅞ3ㅛㅃㅡㅉㅢㅀㅂㄸㄴㅏㄾㄳ"2)ㅀㅝㅍㅢㅁㅃ!ㅖㄵㅃㅑㅜㅈㅙ>66ㄸᴥㅠㅞㅀᴥㅎㄼㄱㄳㅙㅅㅣㅣ ㅟㅋㅃㅡㄾㅞㅉ4ㅐ